In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error,mean_absolute_percentage_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import plotly.express as px

In [30]:
df = pd.read_csv('datasets/Ibovespa4anos.csv',parse_dates=['Data'])
display(df.head(20))

C:\Users\rpjun\AppData\Local\Temp\ipykernel_12880\451688021.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('datasets/Ibovespa4anos.csv',parse_dates=['Data'])


,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2024-05-24,124.306,124.731,125.257,124.259,"9,21M","-0,34%"
1,2024-05-23,124.729,125.650,125.665,124.431,"9,99M","-0,73%"
2,2024-05-22,125.650,127.412,127.412,125.524,"12,40M","-1,38%"
3,2024-05-21,127.412,127.754,128.272,127.205,"9,14M","-0,27%"
4,2024-05-20,127.751,128.151,128.730,127.488,"9,34M","-0,31%"
5,2024-05-17,128.151,128.280,128.464,127.696,"10,26M","-0,10%"
6,2024-05-16,128.284,128.029,128.965,127.922,"9,86M","0,20%"
7,2024-05-15,128.028,128.514,128.646,127.029,"10,86M","-0,38%"
8,2024-05-14,128.515,128.155,128.965,127.962,"11,89M","0,28%"
9,2024-05-13,128.155,127.600,128.669,127.599,"8,92M","0,44%"


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Data      1094 non-null   datetime64[ns]
 1   Último    1094 non-null   float64       
 2   Abertura  1094 non-null   float64       
 3   Máxima    1094 non-null   float64       
 4   Mínima    1094 non-null   float64       
 5   Vol.      1094 non-null   object        
 6   Var%      1094 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 60.0+ KB


In [32]:
df_new = df.copy().drop(columns='Var%')

In [33]:
selecao = df_new['Vol.'].str.contains('K')
df_new[selecao]

,Data,Último,Abertura,Máxima,Mínima,Vol.


In [34]:
df_new.rename(columns={'Data':'ds','Último':'y','Abertura':'abertura','Máxima':'maxima','Mínima':'minima','Vol.':'volume'}, inplace=True)

In [35]:
df_new.head()

,ds,y,abertura,maxima,minima,volume
0,2024-05-24,124.306,124.731,125.257,124.259,"9,21M"
1,2024-05-23,124.729,125.650,125.665,124.431,"9,99M"
2,2024-05-22,125.650,127.412,127.412,125.524,"12,40M"
3,2024-05-21,127.412,127.754,128.272,127.205,"9,14M"
4,2024-05-20,127.751,128.151,128.730,127.488,"9,34M"


In [39]:
df_new['volume'].fillna('0', inplace=True)

In [41]:
df_new['volume'] = df_new['volume'].str.replace('M','',regex=False).str.replace(',','.',regex=False).astype(float)

In [42]:
df_new['volume'] = df_new['volume'] * 1_000_000

In [43]:
df_new.head()

,ds,y,abertura,maxima,minima,volume
0,2024-05-24,124.306,124.731,125.257,124.259,9210000.0
1,2024-05-23,124.729,125.650,125.665,124.431,9990000.0
2,2024-05-22,125.650,127.412,127.412,125.524,12400000.0
3,2024-05-21,127.412,127.754,128.272,127.205,9140000.0
4,2024-05-20,127.751,128.151,128.730,127.488,9340000.0


In [48]:
lr = LinearRegression()

In [46]:
x = df_new.drop(columns=['ds','y'])
y = df_new['y']

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.8)

In [52]:
lr.fit(x_train,y_train)

LinearRegression()

In [53]:
display(lr.coef_)

display(lr.intercept_)

array([-4.82282162e-01,  6.74723806e-01,  8.10324251e-01,  1.26543022e-08])

-0.34131399142891894

In [68]:
y_pred=lr.predict(x_test)

In [69]:
# training_mae = mean_absolute_error(y_train, lr.predict(x_train))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, lr.predict(x_test))
print("Test MAE:", round(mae, 2))
print("Test R2:", round(r2, 2))

Test MAE: 0.48
Test R2: 1.0
